# Génération des graphiques pour la visualisation avec Plotly des données initiales de la CACPL

## Importation des librairies

In [1]:
import pandas as pd
from json import load
from sqlalchemy import create_engine
import plotly.express as px

## Connexion à la BDD

In [2]:
fichierConfig = "../../bdd/config.json"

with open(fichierConfig) as fichier:
    config = load(fichier)["mysql"]
    
engine = create_engine('mysql+' + config["connector"] + '://' + config["user"] + ":" + config["password"] + "@" + config["host"] + ":" + config["port"] + "/" + config["bdd"], echo=True)

## Récupération et préparation des données

In [3]:
requeteCollecteurs = pd.read_sql_query("SELECT * FROM collecteur JOIN tri ON collecteur.idTri = tri.id LEFT JOIN marque ON collecteur.idMarque = marque.id LEFT JOIN categorie ON collecteur.idCategorie = categorie.id;", engine)
requeteDecheteries = pd.read_sql_query("SELECT * FROM decheterie;", engine)

dataCollecteurs = pd.DataFrame(requeteCollecteurs)
dataDecheteries = pd.DataFrame(requeteDecheteries)


# Renommage des colonnes types pour éviter les confusions (et les erreurs)
cols = []
isTri = True
for column in dataCollecteurs.columns:
    if column == "type":
        if isTri:
            cols.append("typeTri")
            isTri = False
        else:
            cols.append("typeCollecteur")
    else:
        cols.append(column)
dataCollecteurs.columns = cols

# Rajout d'une colonne villes par association code INSEE - ville depuis un dictionnaire (pour faciliter les affichages)
dictVilles = { "06029": "Cannes", "06030": "Le Cannet", "06079": "Mandelieu-la-Napoule", "06085": "Mougins", "06138": "Théoule-sur-Mer"}
for i in dataCollecteurs.index:
    dataCollecteurs.loc[i, "ville"] = dictVilles[dataCollecteurs.loc[i, "codeInsee"]]

dataCollecteurs

2021-01-13 21:51:29,576 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-01-13 21:51:29,577 INFO sqlalchemy.engine.base.Engine {}
2021-01-13 21:51:29,580 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-01-13 21:51:29,582 INFO sqlalchemy.engine.base.Engine {}
2021-01-13 21:51:29,586 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-01-13 21:51:29,587 INFO sqlalchemy.engine.base.Engine {}
2021-01-13 21:51:29,590 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-01-13 21:51:29,591 INFO sqlalchemy.engine.base.Engine {}
2021-01-13 21:51:29,593 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-01-13 21:51:29,594 INFO sqlalchemy.engine.base.Engine {}
2021-01-13 21:51:29,595 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM collecteur JOIN tri ON collecteur.idTri = tri.id LEFT JOIN marque ON collecteur.idMarque = marque.id LEFT JO

,objectid,id,volume,quantite,dateInstallation,adresse,adresseComplement,codeInsee,observations,createur,...,idCategorie,idTri,idMarque,id,typeTri,id,nom,id,typeCollecteur,ville
0,1,None,660.0,1,None,Gare tsm,None,06138,None,chpayet,...,1.0,1,1.0,1,om,1.0,Plastic Omnium,1.0,Bac aérien avec cache bac,Théoule-sur-Mer
1,2,None,660.0,3,None,Résidence les terrasses de saint honorat,None,06138,None,chpayet,...,1.0,1,1.0,1,om,1.0,Plastic Omnium,1.0,Bac aérien avec cache bac,Théoule-sur-Mer
2,3,None,660.0,1,None,Prince des iles,None,06138,None,chpayet,...,2.0,1,1.0,1,om,1.0,Plastic Omnium,2.0,Bac aérien sans cache bac,Théoule-sur-Mer
3,4,None,660.0,1,None,Avenue de lerins trottoir de l eglise,None,06138,None,chpayet,...,2.0,1,1.0,1,om,1.0,Plastic Omnium,2.0,Bac aérien sans cache bac,Théoule-sur-Mer
4,5,None,660.0,1,None,Avenue de lerins angle traverse magasin souvenirs,None,06138,None,chpayet,...,2.0,1,1.0,1,om,1.0,Plastic Omnium,2.0,Bac aérien sans cache bac,Théoule-sur-Mer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,963,None,NaN,1,None,1102 avenue du maréchal juin,None,06085,None,chpayet,...,NaN,5,NaN,5,vêtement,NaN,None,NaN,None,Mougins
963,964,None,NaN,1,None,Chemin de la plaine,None,06085,None,chpayet,...,NaN,5,NaN,5,vêtement,NaN,None,NaN,None,Mougins
964,965,None,NaN,1,None,Chemin du refuge,None,06085,None,chpayet,...,NaN,5,NaN,5,vêtement,NaN,None,NaN,None,Mougins
965,966,None,NaN,3,None,Avenue lamartine,None,06085,None,chpayet,...,NaN,5,NaN,5,vêtement,NaN,None,NaN,None,Mougins


## Affichage des graphiques

In [4]:
# Jeu de couleurs de la CACPL
dictVillesCouleurs = {"Cannes": "#7bc3b7", "Le Cannet": "#528238", "Mandelieu-la-Napoule": "#fab55a", "Mougins": "#e84e2c", "Théoule-sur-Mer": "#507999"}
# Jeu de couleurs utilisé pour les icones des collecteurs de la carte
dictTrisCouleurs = {"om": "#bf3932", "papier": "#3e8ecc", "emballage": "#c0b132", "verre": "#35c032", "vêtement": "#31a9be"}

In [5]:
fig = px.histogram(dataCollecteurs, x="ville", color="ville", width=900, color_discrete_map=dictVillesCouleurs, title="Nombre de sites de collecteurs par ville")
fig.update_xaxes(categoryorder="total descending")

In [6]:
# Utilisation de l'histogramme
# Ne marche pas si des valeurs non définies
fig = px.histogram(dataCollecteurs, x="ville", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par ville")
fig.update_xaxes(categoryorder="total descending")

In [7]:
# Utilisation du diagramme à barres
dataCollecteursQuantiteVille = dataCollecteurs.groupby(by=["ville"])["quantite"].sum().reset_index(name="quantite")

fig = px.bar(dataCollecteursQuantiteVille, x="ville", y="quantite", color="ville", width=900, color_discrete_map=dictVillesCouleurs, title="Nombre de collecteurs par ville")
fig.update_xaxes(categoryorder="total descending")

In [8]:
dataSitesCollecteursQuantiteMarque = dataCollecteurs.groupby(by=["nom"])["quantite"].count().reset_index()

fig = px.bar(dataSitesCollecteursQuantiteMarque, x="nom", y="quantite", color="nom", width=900, title="Nombre de sites de collecteurs par marque")
fig.update_xaxes(categoryorder="total descending")

In [9]:
fig = px.histogram(dataCollecteurs, x="nom", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par marque et par ville")
fig.update_xaxes(categoryorder="total descending")

In [10]:
fig = px.histogram(dataCollecteurs, x="nom", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par marque et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [11]:
dataCollecteursQuantiteMarque = dataCollecteurs.groupby(by=["nom"])["quantite"].sum().reset_index(name="quantite")

fig = px.bar(dataCollecteursQuantiteMarque, x="nom", y="quantite", color="nom", width=900, title="Nombre de collecteurs par marque")
fig.update_xaxes(categoryorder="total descending")

In [12]:
fig = px.histogram(dataCollecteurs, x="nom", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par marque et par ville")
fig.update_xaxes(categoryorder="total descending")

In [13]:
fig = px.histogram(dataCollecteurs, x="nom", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par marque et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [14]:
dataSitesCollecteursQuantiteVolume = dataCollecteurs.groupby(by=["volume"]).count().reset_index()

px.bar(dataSitesCollecteursQuantiteVolume, x="volume", y="quantite", color="volume", width=900, title="Nombre de sites de collecteurs par volume (en L)")

In [15]:
fig = px.histogram(dataCollecteurs, x="volume", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par volume (en L) et par ville")
fig.update_xaxes(categoryorder="total descending")

In [16]:
fig = px.histogram(dataCollecteurs, x="volume", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par volume (en L) et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [17]:
fig = px.bar(dataSitesCollecteursQuantiteVolume, x="volume", y="quantite", color="volume", width=900, title="Nombre de sites de collecteurs par volume (en L)")
fig.update_xaxes(type="category")

In [18]:
dataSitesCollecteursQuantiteVolumeVille = dataCollecteurs.groupby(by=["volume", "ville"]).count().reset_index()

In [19]:
fig = px.bar(dataSitesCollecteursQuantiteVolumeVille, x="volume", y="quantite", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par volume (en L)")
fig.update_xaxes(type="category")

In [20]:
fig = px.bar(dataSitesCollecteursQuantiteVolumeVille, x="volume", y="quantite", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par volume (en L)")
fig.update_xaxes(type="category")
fig.update_layout(barmode='group')

In [21]:
dataCollecteursQuantiteVolume = dataCollecteurs.groupby(by=["volume"])["quantite"].sum().reset_index(name="quantite")

In [22]:
px.bar(dataCollecteursQuantiteVolume, x="volume", y="quantite", color="volume", width=900, title="Nombre de collecteurs par volume (en L)")

In [23]:
dataCollecteursQuantiteVolumeVille = dataCollecteurs.groupby(by=["volume", "ville"])["quantite"].sum().reset_index(name="quantite")

In [24]:
fig = px.bar(dataCollecteursQuantiteVolumeVille, x="volume", y="quantite", color="ville", width=900, color_discrete_map=dictVillesCouleurs, title="Nombre de collecteurs par volume (en L) et par ville")
fig.update_xaxes(categoryorder="total descending")

In [25]:
fig = px.bar(dataCollecteursQuantiteVolumeVille, x="volume", y="quantite", color="ville", width=900, color_discrete_map=dictVillesCouleurs, title="Nombre de collecteurs par volume (en L) et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [26]:
fig = px.bar(dataCollecteursQuantiteVolume, x="volume", y="quantite", color="volume", width=900, title="Nombre de collecteurs par volume (en L)")
fig.update_xaxes(type="category")

In [27]:
fig = px.bar(dataCollecteursQuantiteVolumeVille, x="volume", y="quantite", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par volume (en L)")
fig.update_xaxes(type="category")

In [28]:
fig = px.bar(dataCollecteursQuantiteVolumeVille, x="volume", y="quantite", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par volume (en L)")
fig.update_xaxes(type="category")
fig.update_layout(barmode='group')

In [29]:
dataSitesCollecteursQuantiteType = dataCollecteurs.groupby(by=["typeCollecteur"])["quantite"].count().reset_index()

fig = px.bar(dataSitesCollecteursQuantiteType, x="typeCollecteur", y="quantite", color="typeCollecteur", width=900, title="Nombre de sites de collecteurs par type de collecteur")
fig.update_xaxes(categoryorder="total descending")

In [30]:
fig = px.histogram(dataCollecteurs, x="typeCollecteur", color="ville", width=900, color_discrete_map=dictVillesCouleurs, title="Nombre de sites de collecteurs par type de collecteur et par ville")
fig.update_xaxes(categoryorder="total descending")

In [31]:
fig = px.histogram(dataCollecteurs, x="typeCollecteur", color="ville", width=900, color_discrete_map=dictVillesCouleurs, title="Nombre de sites de collecteurs par type de collecteur et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [32]:
dataCollecteursQuantiteType = dataCollecteurs.groupby(by=["typeCollecteur"])["quantite"].sum().reset_index(name="quantite")

fig = px.bar(dataCollecteursQuantiteType, x="typeCollecteur", y="quantite", color="typeCollecteur", width=900, title="Nombre de collecteurs par type de collecteur")
fig.update_xaxes(categoryorder="total descending")

In [33]:
fig = px.histogram(dataCollecteurs, x="typeCollecteur", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par type de collecteur et par ville")
fig.update_xaxes(categoryorder="total descending")

In [34]:
fig = px.histogram(dataCollecteurs, x="typeCollecteur", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par type de collecteur et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [35]:
fig = px.histogram(dataCollecteurs, x="typeTri", color="typeTri", width=900, title="Nombre de sites de collecteurs par type de tri")
fig.update_xaxes(categoryorder="total descending")

In [36]:
fig = px.histogram(dataCollecteurs, x="typeTri", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par type de tri et par ville")
fig.update_xaxes(categoryorder="total descending")

In [37]:
fig = px.histogram(dataCollecteurs, x="typeTri", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de sites de collecteurs par type de tri et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [38]:
fig = px.histogram(dataCollecteurs, x="typeTri", y="quantite", histfunc="sum", color="typeTri", color_discrete_map=dictTrisCouleurs, width=900, title="Nombre de collecteurs par type de tri")
fig.update_xaxes(categoryorder="total descending")

In [39]:
fig = px.histogram(dataCollecteurs, x="typeTri", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par type de tri et par ville")
fig.update_xaxes(categoryorder="total descending")

In [40]:
fig = px.histogram(dataCollecteurs, x="typeTri", y="quantite", histfunc="sum", color="ville", color_discrete_map=dictVillesCouleurs, width=900, title="Nombre de collecteurs par type de tri et par ville")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [41]:
fig = px.histogram(dataCollecteurs, x="ville", color="typeTri", color_discrete_map=dictTrisCouleurs, width=900, title="Nombre de sites de collecteurs par ville et par type de tri")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [42]:
fig = px.histogram(dataCollecteurs, x="ville", y="quantite", histfunc="sum", color="typeTri", color_discrete_map=dictTrisCouleurs, width=900, title="Nombre de collecteurs par ville et par type de tri")
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')

In [43]:
px.pie(dataCollecteurs, names="ville", color="ville", color_discrete_map=dictVillesCouleurs, title="Répartition des sites de collecteurs par ville")

In [44]:
px.pie(dataCollecteursQuantiteVille, values="quantite", names="ville", color="ville", color_discrete_map=dictVillesCouleurs, title="Répartition des collecteurs par ville")

In [45]:
px.pie(dataCollecteurs, names="typeTri", color="typeTri", color_discrete_map=dictTrisCouleurs, title="Répartition des sites de collecteurs par type de tri")

In [46]:
dataCollecteursQuantiteTri = dataCollecteurs.groupby(by=["typeTri"])["quantite"].sum().reset_index(name="quantite")

px.pie(dataCollecteursQuantiteTri, values="quantite", names="typeTri", color="typeTri", color_discrete_map=dictTrisCouleurs, title="Répartition des collecteurs par type de tri")

In [47]:
px.sunburst(dataCollecteurs, path=["ville", "typeTri"], color="ville", color_discrete_map=dictVillesCouleurs, title="Répartition des sites de collecteurs par ville et par type de tri")

In [48]:
px.sunburst(dataCollecteurs, path=["typeTri", "ville"], color="typeTri", color_discrete_map=dictTrisCouleurs, title="Répartition des sites de collecteurs par type de tri et par ville")

In [49]:
px.sunburst(dataCollecteurs, path=["ville", "typeTri"], values="quantite", color="ville", color_discrete_map=dictVillesCouleurs, title="Répartition des collecteurs par ville et par type de tri")

In [50]:
px.sunburst(dataCollecteurs, path=["typeTri", "ville"], values="quantite", color="typeTri", color_discrete_map=dictTrisCouleurs, title="Répartition des collecteurs par type de tri et par ville")